# 决策树ID3学习笔记

## 一、决策树简介



## 二、流程

![伪代码](../imgs/决策树基本流程.png)

## 三、划分选择

&nbsp;&nbsp;&nbsp;&nbsp;随着划分过分不断进行，决策树的分支结点所包含的样本尽可能属于同一类别，即每个结点的"纯度"越来越高（都属于同一类型）。

### 3.1 ID3方法

### 3.1.1 信息增益

&nbsp;&nbsp;&nbsp;&nbsp;信息熵是度量样本集合纯度的一种指标。假设，样本D有K种分类，其中第k中分类的概率为 $p_{k}$ ，那么样本D的信息熵定义为

$$ Ent(D) = -\sum_{k=1}^{K} p_{k}\log_{2} p_{k} $$

其中 $ Ent(D) $ 越小，那么样本D的纯度越高（信息熵越大，那么存在的信息就越多，可以理解为样本D中可分类的空间就越大）。

假设：离散属性a有 $ V $ 个可能值 $ \{ a^{1}, a^{2}, ..., a^{V}\} $ ，那么其中第 $ v $ 个包含的样本为 $ D^{v} $ ，可得 $ D^{v} $ 的信息熵为 $ Ent(D^{v}) $。

在考虑，不同分支下，数据量的不同，赋予分支数据量权重 $ |D^{v}| / |D| $ ，那么对样本D进行a划分的信息增益为：

$$ Gain(D, a) = Ent(D) - \sum_{v=1}^{V} \frac{|D^{v}|}{|D|} Ent(D^{v}). $$

一般地，若 $ Gain(D, a) $ 信息增益越大，那么以 $ a $ 来进行划分的纯度提升越大。

### 3.1.2 计算步骤

&nbsp;&nbsp;&nbsp;&nbsp;假设：数据集如下图所示

![3.1.2数据集](../imgs/西瓜数据集2.png)

1. 首先，样本对应的信息熵，其中正例8例、反例9例，总共17例。$ Ent(D) = -\sum_{k=1}^{2} p_{k}\log_{2} p_{k}\ = - (\frac{8}{17} \log_{2} \frac{8}{17} + \frac{9}{17} \log_{2} \frac{9}{17}) = 0.998 $

2. 若以色泽为例，进行划分，色泽包括 { '青绿', '乌黑', '浅白'}，那么需要计算三个信息熵，最终可得信息增益为0.109.
3. 分别计算其他属性的划分，对应的信息增益，最终纹理属性对应的 信息增益最大 为 0.381。
4. 以纹理作为划分，得到3个新样本集，再对3个新样本集分别重复1-3步操作。
5. 直至无法划分出新的新样本集，算法结束。

### 3.1.3 缺点总结

ID3算法虽然提出了新思路，但是还是有很多值得改进的地方。　　

1. ID3没有考虑连续特征，比如长度，密度都是连续值，无法在ID3运用。这大大限制了ID3的用途。

2. ID3采用信息增益大的特征优先建立决策树的节点。很快就被人发现，在相同条件下，取值比较多的特征比取值少的特征信息增益大。比如一个变量有2个值，各为1/2，另一个变量为3个值，各为1/3，其实他们都是完全不确定的变量，但是取3个值的比取2个值的信息增益大。如果校正这个问题呢？

3. ID3算法对于缺失值的情况没有做考虑

4. 没有考虑过拟合的问题

　　　　ID3 算法的作者昆兰基于上述不足，对ID3算法做了改进，这就是C4.5算法，也许你会问，为什么不叫ID4，ID5之类的名字呢?那是因为决策树太火爆，他的ID3一出来，别人二次创新，很快 就占了ID4， ID5，所以他另辟蹊径，取名C4.0算法，后来的进化版为C4.5算法。下面我们就来聊下C4.5算法


In [7]:
import numpy as np
import pandas as pd

import sys
sys.path.append('../')
from utils.dataset import load_watermelon_2

from collections import Counter


def ent(y: np.array) -> float:
    """获取样本集的信息熵

    Args:
        y (np.array): [标签集合]

    Returns:
        float: [信息熵]
    """
    target_counter = Counter(y)
    target_number = y.shape[0]

    ent_value = 0
    for _, key in enumerate(target_counter.keys()):
        _ent = -1 * (target_counter[key] / target_number) * (np.log2(target_counter[key] / target_number))
        ent_value +=_ent
    return ent_value


def gain(d_number: int, d_ent: float, df: pd.DataFrame, a: str) -> float:
    """获取信息增益

    Args:
        d_number (int): [整体样本数量]
        d_ent (float): [整体样本的信息熵]
        df (pd.DataFrame): [结点样本]]
        a (str): [划分属性a]

    Returns:
        float: [节点样本 对应 a属性划分的信息增益]
    """
    a_unique_values = df[a].unique()

    dv_ent_value = 0
    for a_value in a_unique_values:
        v_df = df.loc[df[a]==a_value]
        weight = v_df.shape[0] / d_number
        dv_ent = weight * ent(v_df['target'])
        dv_ent_value += dv_ent
    
    # gain
    return d_ent - dv_ent_value

# ------------------------------------------------------


class ID3(object):
    def __init__(self, df) -> None:
        self.df = df
        self.shape = df.shape
        self.tree = list()
        self.d_ent = ent(self.df['target'])
        super().__init__()
    
    def separate(self, df, parent='root'):
        """进行划分

        Args:
            df ([type]): [节点样本]
            parent (str, optional): [该节点对应的父节点名称]. Defaults to 'root'.

        Returns:
            [type]: [若无数据或数据只有一个target值时，返回1]
        """
        X_df = df.loc[:, ['色泽', '根蒂', '敲声', '纹理', '脐部', '触感']]
        y = df['target']

        if X_df.shape[0] == 0:
            # 若无数据输入
            return 1
        elif y.nunique() == 1:
            # 若多类型
            return 1

        max_gain = 0
        res_dict = {}
        for _, col in enumerate(X_df.columns):  # 对每个属性进行划分
            gain_a = gain(
                self.shape[0], self.d_ent, df, col
            )
            if gain_a > max_gain:
                max_gain = gain_a
                res_dict['gain'] = gain_a
                res_dict['a'] = col
                res_dict['parent'] = parent
        
        # 继续划分
        a_unique_values = df[res_dict['a']].unique()
        for a_value in a_unique_values:
            v_df = df.loc[df[res_dict['a']]==a_value]
            res_dict['data'] = list(v_df.index)
            res_dict['a_v'] = res_dict['a'] + '-' + a_value

            # update self.tree
            rv = self.separate(v_df, parent=res_dict['a'] + '-' + a_value)
            if rv == 1:
                self.tree.append({'a': res_dict['a'], 'parent': res_dict['parent'], 'counter': dict(Counter(v_df['target']))})
    
    def run(self, ):
        """ID3划分选择

        Returns:
            [list]: [划分的顺序关系]
        """
        self.separate(self.df)
        return self.tree



In [8]:
watermelon_df = load_watermelon_2()
X = watermelon_df.loc[:, ['色泽', '根蒂', '敲声', '纹理', '脐部', '触感']]
y = watermelon_df['target']

separation_res = ID3(watermelon_df).run()
print("划分规则 连接关系 -----> ")
print(separation_res)

划分规则 连接关系 -----> 
[{'a': '根蒂', 'parent': '纹理-清晰', 'counter': {1: 5}}, {'a': '色泽', 'parent': '根蒂-稍蜷', 'counter': {1: 1}}, {'a': '触感', 'parent': '色泽-乌黑', 'counter': {1: 1}}, {'a': '触感', 'parent': '色泽-乌黑', 'counter': {0: 1}}, {'a': '根蒂', 'parent': '纹理-清晰', 'counter': {0: 1}}, {'a': '触感', 'parent': '纹理-稍糊', 'counter': {1: 1}}, {'a': '触感', 'parent': '纹理-稍糊', 'counter': {0: 4}}, {'a': '纹理', 'parent': 'root', 'counter': {0: 3}}]
